## Завдання 2

## Виправимо затримку із додаванням в базу даних

``` mysql 
UPDATE marketing_cost
SET date_column = DATE_SUB(install_date, INTERVAL 1 DAY)
WHERE platfrom = 'iOS' AND media_source = 'applovin_int';
```

## Запит для звіту

``` mysql 
SELECT 
    ua.install_date, 
    ua.platform, 
    ua.media_source,
    SUM(CASE WHEN DATEDIFF(ur.event_time, ua.install_time) = 0 THEN ur.amount ELSE 0 END) / mc.cost AS ROAS_d0,
    SUM(CASE WHEN DATEDIFF(ur.event_time, ua.install_time) = 0 AND ua.install_time BETWEEN DATE_SUB(mc.install_date, INTERVAL 6 DAY) AND mc.install_date THEN ur.amount ELSE 0 END) / SUM(CASE WHEN ua.install_time BETWEEN DATE_SUB(mc.install_date, INTERVAL 6 DAY) AND mc.install_date THEN mc.cost ELSE 0 END) AS ROAS_d0_weekly
FROM 
    user_attribute AS ua
JOIN 
    user_revenue AS ur ON ua.user_id = ur.user_id
JOIN 
    marketing_cost AS mc ON ua.platform = mc.platform AND ua.media_source = mc.media_source AND DATE(ua.install_time) = mc.install_date
WHERE 
    MONTH(ua.install_time) = 6 AND YEAR(ua.install_time) = YEAR(CURRENT_DATE) 
GROUP BY 
    ua.install_date, ua.platform, ua.media_source
HAVING 
    ROAS_d0_weekly > 30
ORDER BY 
    ua.install_date, ua.platform, ua.media_source;
```

### ROAS_d0
``` mysql SUM(CASE WHEN DATEDIFF(ur.event_time, ua.install_time) = 0 THEN ur.amount ELSE 0 END) / mc.cost: ```


Тут ми обчислюємо суму платежів (ur.amount), які відбулися в той же день, що і встановлення 
```mysql (DATEDIFF(ur.event_time, ua.install_time) = 0), ```
і ділимо цю суму на витрати на рекламу (mc.cost) для цієї групи.

### ROAS_d0_weekly
Тут ми робимо суму платежів за ті ж умови, але також враховуємо, що встановлення застосунку відбулося протягом попередніх 7 днів

``` mysql (ua.install_time BETWEEN DATE_SUB(mc.install_date, INTERVAL 6 DAY) AND mc.install_date) ```

##  Загалом, цей запит виконує наступне:
Приєднує таблиці user_attribute, user_revenue, і marketing_cost за спільними ключами.

Враховує транзакції, що відбулись в першу добу після встановлення.

Групує дані за install_date, platform, і media_source.

Обчислює ROAS_d0 і ROAS_d0_weekly для кожної групи.

Вибирає лише ті записи, де ROAS_d0_weekly більше за 30.

Сортує результати за датою установки, платформою та джерелом трафіку.